In [71]:
# import requiered libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [73]:
!git branch -a

* QC
  feature/extraxt-patient-data
  main
  update/extract-patient-data
  remotes/origin/HEAD -> origin/main
  remotes/origin/feature/extraxt-patient-data
  remotes/origin/main
  remotes/origin/ralf/add-data-overview
  remotes/origin/update/extract-patient-data


In [75]:
# Read raw train data
train = pd.read_csv( '../data/raw/train.csv' )
train.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [76]:
# Define function to run comparisons
def compare_data( id , metric ):
    # Filter the raw DataFrame
    df_raw = train[ train['p_num'] == id ]
    
    # read the parsed data for the given patient ID
    df_parsed = pd.read_csv( f'../data/interim/{id}_train.csv' )
    
    # generate data frames of unique non-NA values
    val1_df = pd.DataFrame({
        'group':'raw',
        'vals':df_raw[df_raw.columns[df_raw.columns.str.startswith(f'{metric}-')]].melt().value.unique()
    })
    
    val2_df = pd.DataFrame({
        'group':'processed',
        'vals':df_parsed[metric].unique()
    })

    val1_df.sort_values( 'vals', inplace = True )
    val2_df.sort_values( 'vals', inplace = True )
    
    # Calculate differences
    vals_df = pd.concat([val1_df,val2_df],axis = 0 )
    vals_df = vals_df.groupby('group').agg(['mean','std',np.min,np.max])
    vals_df = pd.DataFrame( vals_df )
    checks_df  = pd.DataFrame( vals_df.loc['raw',:]-vals_df.loc['processed',:] )
    checks_df.rename( columns = {0:'diff'} , inplace  =True )
    checks_df['p_id'] = id
    checks_df['diff'] = abs( checks_df['diff'] )
    checks_df.reset_index(inplace=True)
    checks_df.drop('level_0',axis=1,inplace=True)
    checks_df.rename( columns = {'level_1':'metric'} , inplace = True )
    checks_df['feature'] =  metric
    checks_df = checks_df[['p_id','feature','metric','diff']]
    checks_df['diff_check'] = checks_df['diff'] < 1e-6

    # Return final DataFrames
    return checks_df , val1_df , val2_df , df_raw , df_parsed
    

In [79]:
# List of ids to check
ids = ['p01','p02','p03','p04','p05','p06','p10','p11','p12']
metrics = ['bg','insulin','carbs','hr','steps','cals']

# Instantiate an empty list to store results
res_l = list()

# Run a For loop
for id in ids:
    for metric in metrics:
        # print(id,metric) 
        checks_df , _ , _  , _ , _ = compare_data( id , metric )
        res_l.append( checks_df )

In [80]:
# Join the individual DataFrames
res_df = pd.concat( res_l ).drop('diff_check',axis=1)
pivot_df = pd.pivot( res_df , index = ['p_id','feature'] , columns = 'metric' , values ='diff' )
pivot_df.reset_index( inplace = True )
pivot_df = pivot_df[['p_id','feature','mean','std','min','max']]
pivot_df

metric,p_id,feature,mean,std,min,max
0,p01,bg,0.000000,0.000000,0.0,0.0
1,p01,cals,0.000000,0.000000,0.0,0.0
2,p01,carbs,0.000000,0.000000,0.0,0.0
3,p01,hr,0.000000,0.000000,0.0,0.0
4,p01,insulin,0.000000,0.000000,0.0,0.0
5,p01,steps,0.000000,0.000000,0.0,0.0
6,p02,bg,0.000000,0.000000,0.0,0.0
7,p02,cals,0.000000,0.000000,0.0,0.0
8,p02,carbs,0.000000,0.000000,0.0,0.0
9,p02,hr,0.000000,0.000000,0.0,0.0


In [85]:
id = 'p11'
metric = 'cals'

In [87]:
df_raw = train[ train['p_num'] == id ]
df_parsed = pd.read_csv( f'../data/interim/{id}_train.csv' )

In [99]:
print( (df_raw==15.36).any().any() )
print( (df_parsed==15.36).any().any() )

True
False


In [39]:
# Check specifically on p12 insulin
checks_df , val1_df , val2_df , df_raw , df_parsed = compare_data( 'p11' , 'cals' )
print(val1_df.vals.mean())
print(val2_df.vals.mean())
print( val1_df.shape )
print( val2_df.shape )

20.039764468371466
20.042915824915823
(1487, 2)
(1486, 2)


In [33]:
set(val1_df['vals']) - set(val2_df['vals'])

{15.36, nan}

In [69]:
dfs = df_raw[df_raw.columns[df_raw.columns.str.startswith(f'cals-')]] 
row = np.where( (dfs == 15.36).sum( axis  = 1 ) >0 )[0]
col = np.where( (dfs == 15.36).sum( axis  = 0 ) >0 )[0]
dfs.iloc[row,col]

,cals-5:55,cals-5:50,cals-5:45,cals-5:40,cals-5:35,cals-5:30,cals-5:25,cals-5:20,cals-5:15,cals-5:10,...,cals-2:20,cals-2:15,cals-2:10,cals-2:05,cals-2:00,cals-1:55,cals-1:50,cals-1:45,cals-1:40,cals-1:35
131477,5.95,6.07,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,...,5.95,5.95,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36
131478,6.07,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,...,5.95,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61
131479,5.95,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,...,6.07,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27
131480,5.95,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,...,6.07,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47
131481,6.07,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,...,6.07,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00
131482,5.95,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,...,8.69,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19
131483,5.95,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,...,5.95,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39
131484,5.95,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,...,6.43,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88
131485,5.95,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,5.95,...,6.19,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88,6.91
131486,5.95,5.95,6.07,6.07,5.95,6.07,5.95,5.95,5.95,5.95,...,15.36,12.61,7.27,15.47,15.00,11.19,19.39,19.88,6.91,6.43
